In [113]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os
import cohere

In [114]:
load_dotenv(override=True)

gemini_api_key = os.getenv("GEMINI_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")

if gemini_api_key:
    print(f"Gemini API Key exists and begins with {gemini_api_key[:8]}")
else:
    print("Gemini API Key does not exist")

if cohere_api_key:
    print(f"Cohere API Key exists and begins with {cohere_api_key[:8]}")
else:
    print("Cohere API Key does not exist")

Gemini API Key exists and begins with AIzaSyAy
Cohere API Key exists and begins with SLYqRAFo


In [115]:
MODEL = os.getenv("GEMINI_MODEL")
client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

In [116]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [117]:
print(linkedin)

   
Contact
anandjain2507@gmail.com
www.linkedin.com/in/anandjain25
(LinkedIn)
Top Skills
Large Language Models (LLM)
GEN AI
Lambda Expressions
Certifications
AWS SysOps Associate 2022:
Storage & Data Classification
AWS SysOps Associate 2022:
Firewall Solutions
HTML5 with JavaScript & CSS3:
HTML5 & Javascript
Node.js
DockerAnand Jain
Amazon Certified AWS Developer-Associate | Node.js | Express.js |
Javascript | LLM Engineering | Elasticsearch | Jest | Mocha | Docker
| Git | SVN | AWS | GenAI | Camunda BPMN | AI Prompts
Pune, Maharashtra, India
Summary
First and foremost, I love writing code. Ever since writing my first
program in Java and manipulating it to produce the desired output.
I have been obsessed with programming languages and how they
work behind the scenes. I believe in the power of programming to
improve the lives of people around the world.
This passion carried me through my education at Medi-caps. I
studied new programming languages, algorithms, frameworks, and
platforms,

In [118]:
reader = PdfReader("me/linkedin.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [119]:
print(resume)

   
Contact
anandjain2507@gmail.com
www.linkedin.com/in/anandjain25
(LinkedIn)
Top Skills
Large Language Models (LLM)
GEN AI
Lambda Expressions
Certifications
AWS SysOps Associate 2022:
Storage & Data Classification
AWS SysOps Associate 2022:
Firewall Solutions
HTML5 with JavaScript & CSS3:
HTML5 & Javascript
Node.js
DockerAnand Jain
Amazon Certified AWS Developer-Associate | Node.js | Express.js |
Javascript | LLM Engineering | Elasticsearch | Jest | Mocha | Docker
| Git | SVN | AWS | GenAI | Camunda BPMN | AI Prompts
Pune, Maharashtra, India
Summary
First and foremost, I love writing code. Ever since writing my first
program in Java and manipulating it to produce the desired output.
I have been obsessed with programming languages and how they
work behind the scenes. I believe in the power of programming to
improve the lives of people around the world.
This passion carried me through my education at Medi-caps. I
studied new programming languages, algorithms, frameworks, and
platforms,

In [120]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [121]:
print(summary)

My name is Anand Jain. I'm an technical lead, software engineer and data scientist. I'm originally from Indore, India, but I moved to Pune in 2018.
I love all foods, particularly Italian food, but strangely I'm love by almost all forms of cheese. I'm not allergic, I just love the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.


In [122]:
name = "Anand Jain"

In [123]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume Content:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [124]:
system_prompt

"You are acting as Anand Jain. You are answering questions on Anand Jain's website, particularly questions related to Anand Jain's career, background, skills and experience. Your responsibility is to represent Anand Jain for interactions on the website as faithfully as possible. You are given a summary of Anand Jain's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Anand Jain. I'm an technical lead, software engineer and data scientist. I'm originally from Indore, India, but I moved to Pune in 2018.\nI love all foods, particularly Italian food, but strangely I'm love by almost all forms of cheese. I'm not allergic, I just love the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nanandjai

In [125]:
def chat(message, history):
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [126]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [127]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel


class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [128]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume Content:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [129]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = (
        f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    )
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [130]:
# ollama_deepseek = OpenAI(
#     api_key="ollama",
#     base_url="http://localhost:11434/v1"
# )

In [131]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]
    response = client.beta.chat.completions.parse(
        model=os.getenv("GEMINI_MODEL"), messages=messages, response_format=Evaluation
    )
    return response.choices[0].message.parsed

In [132]:
messages = [{"role": "system", "content": system_prompt}] + [
    {"role": "user", "content": "do you hold a patent?"}
]
response = client.chat.completions.create(model=MODEL, messages=messages)
reply = response.choices[0].message.content

In [133]:
reply

"That's a great question! Based on the information I have available about my professional background and experience, I do not currently hold a patent.\n\nI'm always learning and exploring new areas in software development and technology, but patents are not something I've pursued so far."

In [134]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The answer is good. It's professional and answers the question directly. It would be good if the agent elaborated on what they are exploring in software development and technology. This would be more engaging.")

In [135]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = (
        system_prompt
        + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    )
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = (
        [{"role": "system", "content": updated_system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [136]:
def chat(message, history):
    if "patent" in message:
        system = (
            system_prompt
            + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
        )
    else:
        system = system_prompt
    messages = (
        [{"role": "system", "content": system}]
        + history
        + [{"role": "user", "content": message}]
    )
    response = client.chat.completions.create(model=MODEL, messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [137]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
This response is unacceptable. It's written in pig latin, which is completely unprofessional and does not align with the instructions to be professional and engaging.
Passed evaluation - returning reply
